In [76]:
#%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models

print(tf.version)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


# Helper libraries
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn.utils import compute_class_weight

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from pathlib import Path
import os.path
#import datetime, os

<module 'tensorflow._api.v2.version' from 'c:\\Users\\elec_\\anacon\\anaconda3\\envs\\MLIS2car\\lib\\site-packages\\tensorflow\\_api\\v2\\version\\__init__.py'>
Num GPUs Available:  0


In [77]:
batch_size = 16
img_size = [240, 320]

df = pd.read_csv(r'machine-learning-in-science-ii-2023/training_norm.csv')
df['filename'] = df["image_id"].astype(str) + ".png"
#df['label'] = [df["angle"], df["speed"]]
print(df)


#x_train = df[0:int(len(df) * 0.7)]
x_validate = df[int(len(df) * 0.8):int(len(df) * 0.9)]
x_evaluate = df[int(len(df) * 0.9):]

x_train = df[0:int(len(df) * 0.8)]
#x_validate = df[int(len(df) * 0.8):]

training_images = tf.keras.preprocessing.image_dataset_from_directory(
    "machine-learning-in-science-ii-2023/training_data",
    image_size=img_size,
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=1,
)


       image_id   angle  speed   filename
0             1  0.4375    0.0      1.png
1             2  0.8125    1.0      2.png
2             3  0.4375    1.0      3.png
3             4  0.6250    1.0      4.png
4             5  0.5000    0.0      5.png
...         ...     ...    ...        ...
13788     13794  0.6250    1.0  13794.png
13789     13795  0.4375    1.0  13795.png
13790     13796  0.5625    0.0  13796.png
13791     13797  0.6250    0.0  13797.png
13792     13798  0.6875    1.0  13798.png

[13793 rows x 4 columns]
Found 13798 files belonging to 1 classes.
Using 11039 files for training.


In [78]:
training_datagen = ImageDataGenerator(
    rescale = 1./255,
    fill_mode='nearest')

train_generator = training_datagen.flow_from_dataframe(
    dataframe=x_train,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["angle","speed"],
    target_size=img_size,
    batch_size=16,
    class_mode='other')

validation_datagen = ImageDataGenerator(rescale = 1./255)

val_generator = validation_datagen.flow_from_dataframe(
    dataframe=x_validate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["angle","speed"],
    target_size=img_size,
    class_mode='other')

evaluate_datagen = ImageDataGenerator(rescale = 1./255)

eval_generator = evaluate_datagen.flow_from_dataframe(
    dataframe=x_evaluate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["angle","speed"],
    target_size=img_size,
    class_mode='other')


Found 11034 validated image filenames.
Found 1379 validated image filenames.
Found 1380 validated image filenames.


In [79]:
shape = (*img_size, 3) # inherited image size with 3 color filters

# CNN base
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), strides=(2, 2), activation='relu', input_shape=shape))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
        
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.Conv2D(32, (3, 3),strides=(2, 2), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))


model.add(layers.Conv2D(64, (3, 3), strides=(2, 2), activation='relu'))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.2))
model.add(layers.Flatten()),


model.add(layers.Dense(64, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.4))

model.add(layers.Dense(16, activation='relu'))
model.add(layers.BatchNormalization())

model.add(layers.Dense(8, activation='relu'))
model.add(layers.BatchNormalization())

# Output layer
model.add(layers.Flatten())
model.add(layers.Dense(2,  activation='relu', kernel_initializer='normal'))


In [80]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_57 (Conv2D)          (None, 119, 159, 32)      896       
                                                                 
 batch_normalization_79 (Bat  (None, 119, 159, 32)     128       
 chNormalization)                                                
                                                                 
 max_pooling2d_38 (MaxPoolin  (None, 59, 79, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_58 (Conv2D)          (None, 57, 77, 32)        9248      
                                                                 
 conv2d_59 (Conv2D)          (None, 28, 38, 32)        9248      
                                                                 
 batch_normalization_80 (Bat  (None, 28, 38, 32)     

In [85]:
model.compile(
    optimizer='adam',
    loss='mean_squared_error'
)

history = model.fit(
    train_generator, 
    batch_size=batch_size,
    validation_data=val_generator,
    epochs=5
)

Epoch 1/5
690/690 [==============================] - 101s 144ms/step - loss: 0.0405 - val_loss: 0.0254
Epoch 2/5
690/690 [==============================] - 100s 144ms/step - loss: 0.0304 - val_loss: 0.0233
Epoch 3/5
690/690 [==============================] - 102s 147ms/step - loss: 0.0279 - val_loss: 0.0351
Epoch 4/5
690/690 [==============================] - 99s 143ms/step - loss: 0.0271 - val_loss: 0.0453
Epoch 5/5
690/690 [==============================] - 104s 151ms/step - loss: 0.0239 - val_loss: 0.0227


In [87]:
#model = tf.keras.models.load_model('model.h6')

test_images = tf.keras.preprocessing.image_dataset_from_directory(
    "machine-learning-in-science-ii-2023/test_data",
    image_size=img_size,
    seed=1,
)

normalization_layer = tf.keras.layers.Rescaling(1./255)

test_images = test_images.map(lambda x, y: (normalization_layer(x), y))

import csv


prediction = model.predict(test_images)
print(prediction.shape)
import math

a= prediction[:,0]
b= prediction[:,1]
b+=0.5
b= np.floor(b)

submissiondata={"angle":a , "speed":b} 
df = pd.DataFrame(submissiondata)
df.index += 1 
df.to_csv('Submission.csv')


Found 1020 files belonging to 1 classes.
32/32 [==============================] - 2s 66ms/step
(1020, 2)


In [88]:
from sklearn.metrics import classification_report

x_evaluate = df[int(len(df) * 0.8):]

test_loss = model.evaluate(
    eval_generator,
    verbose=1
)

test_datagen = ImageDataGenerator(rescale = 1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=x_evaluate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    target_size=img_size,
    class_mode=None)



#test_loss

predict_model = model.predict(test_generator, verbose =1)

# y = np.concatenate([y for x, y in x_evaluate], axis=0)
# true = np.argmax(y, axis = 1)

# print(true)
# predict_model = np.argmax(predict_model, axis = 1)
# print(predict_model)

# print(classification_report(true, predict_model, target_names=class_names))


44/44 [==============================] - 9s 197ms/step - loss: 0.0229


KeyError: 'filename'